In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.covariance import LedoitWolf
import matplotlib.pyplot as plt

from plotly.subplots import make_subplots
import plotly.graph_objects as go

### Daily Model
1. Add when there's a signal
2. Potential crash/bear market when there's 2 signals for the past month\

In [5]:
# visualization, not quantifiable
symbol = "^TWII"
data = yf.download(symbol, start='2018-01-01', rounding=2, progress=False)
returns = data[["Close"]].pct_change().dropna()


returns["pct_power_5"] = np.power(returns.Close, 5)
tajm = 242
alpha = 2 / (tajm + 1)
returns['cumsum_bm'] = returns.Close.cumsum()
returns['ma_cumsum_bm'] = returns.cumsum_bm.rolling(60).mean()
returns['pct_for_calculating_profit'] = returns.Close.shift(-2)

returns["pct_power_5_ema"] = returns["pct_power_5"].ewm(alpha=alpha, adjust=False).mean()
returns['pct_power_5_ema_shift_1'] = returns.pct_power_5_ema.shift(1)

returns['signal'] = returns.apply(lambda row: 1 if row['pct_power_5_ema'] > row['pct_power_5_ema_shift_1']*1.01 or row['pct_power_5_ema'] > 0 else -1, axis=1)

returns['strat_pct'] = returns.signal * returns.pct_for_calculating_profit

k = 0
u = -1

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                     subplot_titles=symbol)

fig.add_trace(go.Scatter(
    x=returns.index[k:u],
    y=returns.iloc[k:u].cumsum_bm,
    mode='lines',
    name='Close Price'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=returns.index[k:u],
    y=returns.iloc[k:u]['signal'],
    mode='lines',
    name='Signal'
), row=2, col=1)

fig.update_layout(
    title=symbol,
    template="plotly_white",
    height=600
)

fig.update_xaxes(title_text="Index", row=2, col=1)
fig.update_yaxes(title_text="Signal", row=2, col=1)

fig.show()

### Monthly Model
1. Add to the position when there's a signal with abnormal plunges/dips
2. bear market when there's 2 signals for the past 3 months, reverse when there's abnormal dips

In [ ]:
# visualization, not quantifiable - Monthly Version
symbol = "^TWII"
data = yf.download(symbol, start='2016-01-01', rounding=2, progress=False)
returns = data[["Close"]].pct_change().dropna()
returns = returns.resample("ME").sum()

returns["pct_power_5"] = np.power(returns.Close, 5)
tajm = 3
alpha = 2 / (tajm + 1)
returns['cumsum_bm'] = returns.Close.cumsum()
returns['ma_cumsum_bm'] = returns.cumsum_bm.rolling(60).mean()
returns['pct_for_calculating_profit'] = returns.Close.shift(-2)

returns["pct_power_5_ema"] = returns["pct_power_5"].ewm(alpha=alpha, adjust=False).mean()
returns['pct_power_5_ema_shift_1'] = returns.pct_power_5_ema.shift(1)

returns['signal'] = returns.apply(lambda row: 1 if row['pct_power_5_ema'] > row['pct_power_5_ema_shift_1']*1.01 or row['pct_power_5_ema'] > 0 else -1, axis=1)

returns['strat_pct'] = returns.signal * returns.pct_for_calculating_profit

k = 0
u = -1

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                     subplot_titles=symbol)

fig.add_trace(go.Scatter(
    x=returns.index[k:u],
    y=returns.iloc[k:u].cumsum_bm,
    mode='lines',
    name='Close Price'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=returns.index[k:u],
    y=returns.iloc[k:u]['signal'],
    mode='lines',
    name='Signal'
), row=2, col=1)

fig.update_layout(
    title=symbol,
    template="plotly_white",
    height=600
)

fig.update_xaxes(title_text="Index", row=2, col=1)
fig.update_yaxes(title_text="Signal", row=2, col=1)

fig.show()

### TWII to TWOII

In [330]:
import yfinance as yf
start = '2019-01-01'
twii = yf.download("^TWII", start=start, progress=False, rounding=2)
twoii = yf.download("^TWOII", start=start, progress=False, rounding=2)
df = twii[['Close']].copy()
df.columns = ["twii"]
df['twoii'] = twoii['Close']
df_log = np.log(df / df.shift(1))
df_log = df_log.resample("W-FRI").sum()

In [331]:
import plotly.graph_objs as go
import pandas as pd
import yfinance as yf
start = '2019-01-01'
twii = yf.download("^TWII", start=start, progress=False, rounding=2)
twoii = yf.download("^TWOII", start=start, progress=False, rounding=2)
df = twii[['Close']].copy()
df.columns = ["twii"]
df['twoii'] = twoii['Close']
df_log = np.log(df / df.shift(1))
df_log = df_log.resample("W-FRI").sum()
# Assuming df_log is your DataFrame with the columns 'twii' and 'twoii'
df_log['diff'] = df_log['twii'] - df_log['twoii']
df_cumsum = df_log.cumsum()


import plotly.graph_objs as go

# Create the Plotly figure
fig = go.Figure()

# Add the TWII trace (primary y-axis)
fig.add_trace(go.Scatter(x=df_cumsum.index, y=df_cumsum['twii'], mode='lines', name='TWII'))

# Add the TWOII trace (primary y-axis)
fig.add_trace(go.Scatter(x=df_cumsum.index, y=df_cumsum['twoii'], mode='lines', name='TWOII'))

# Add the Cumulative Sum of the Difference trace (secondary y-axis)
fig.add_trace(go.Scatter(x=df_cumsum.index, y=df_cumsum['diff'], mode='lines', name='Cumulative Sum of Difference', yaxis="y2"))

# Customize the layout
fig.update_layout(
    title="Cumulative Sum of Difference Between TWII and TWOII",
    xaxis_title="Date",
    yaxis_title="TWII / TWOII",
    yaxis2=dict(
        title="Cumulative Sum of Difference",
        overlaying="y",  # Overlay on the primary y-axis
        side="right"     # Place the secondary y-axis on the right
    ),
    template="plotly_dark"
)

# Show the plot
fig.show()


In [348]:
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import yfinance as yf
import numpy as np

symbol = "^TWII"
start = '2018-01-01'
data = yf.download(symbol, start=start, rounding=2, progress=False)

# Data for plot 1
data_plot1 = data[["Close"]].pct_change().dropna()
data_plot1["pct_power_5"] = np.power(data_plot1.Close, 5)
tajm = 242
alpha = 2 / (tajm + 1)
data_plot1['cumsum_bm'] = data_plot1.Close.cumsum()
data_plot1['ma_cumsum_bm'] = data_plot1.cumsum_bm.rolling(60).mean()
data_plot1['pct_for_calculating_profit'] = data_plot1.Close.shift(-2)
data_plot1["pct_power_5_ema"] = data_plot1["pct_power_5"].ewm(alpha=alpha, adjust=False).mean()
data_plot1['pct_power_5_ema_shift_1'] = data_plot1.pct_power_5_ema.shift(1)
data_plot1['signal'] = data_plot1.apply(lambda row: 1 if row['pct_power_5_ema'] > row['pct_power_5_ema_shift_1']*1.01 or row['pct_power_5_ema'] > 0 else -1, axis=1)
data_plot1['strat_pct'] = data_plot1.signal * data_plot1.pct_for_calculating_profit

# Data for plot 2 (monthly)
data_plot2 = data[["Close"]].pct_change().dropna()
data_plot2 = data_plot2.resample("ME").sum()

data_plot2["pct_power_5"] = np.power(data_plot2.Close, 5)
tajm = 3
alpha = 2 / (tajm + 1)
data_plot2['cumsum_bm'] = data_plot2.Close.cumsum()
data_plot2['ma_cumsum_bm'] = data_plot2.cumsum_bm.rolling(60).mean()
data_plot2['pct_for_calculating_profit'] = data_plot2.Close.shift(-2)
data_plot2["pct_power_5_ema"] = data_plot2["pct_power_5"].ewm(alpha=alpha, adjust=False).mean()
data_plot2['pct_power_5_ema_shift_1'] = data_plot2.pct_power_5_ema.shift(1)
data_plot2['signal'] = data_plot2.apply(lambda row: 1 if row['pct_power_5_ema'] > row['pct_power_5_ema_shift_1']*1.01 or row['pct_power_5_ema'] > 0 else -1, axis=1)
data_plot2['strat_pct'] = data_plot2.signal * data_plot2.pct_for_calculating_profit

# Data for plot 3 (TWII vs TWOII)
twoii = yf.download("^TWOII", start=start, progress=False, rounding=2)
data_plot3 = data[['Close']].copy()
data_plot3.columns = ["twii"]
data_plot3['twoii'] = twoii['Close']
df_log = np.log(data_plot3 / data_plot3.shift(1))
df_log = df_log.resample("W-FRI").sum()

df_log['diff'] = df_log['twii'] - df_log['twoii']
df_cumsum = df_log.cumsum()

# Create subplots with reduced vertical spacing
fig = make_subplots(rows=5, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                     subplot_titles=[symbol, symbol, symbol, symbol, symbol])

# Add plot 1
fig.add_trace(go.Scatter(x=data_plot1.index, y=data_plot1.cumsum_bm, mode='lines', name='Close Price'), row=1, col=1)
fig.add_trace(go.Scatter(x=data_plot1.index, y=data_plot1['signal'], mode='lines', name='Signal for plot1'), row=2, col=1)

# Add plot 2
fig.add_trace(go.Scatter(x=data_plot2.index, y=data_plot2.cumsum_bm, mode='lines', name='Close Price'), row=3, col=1)
fig.add_trace(go.Scatter(x=data_plot2.index, y=data_plot2['signal'], mode='lines', name='Signal for plot2'), row=4, col=1)

# Add plot 3
fig.add_trace(go.Scatter(x=df_cumsum.index, y=df_cumsum['twii'], mode='lines', name='TWII'), row=5, col=1)
fig.add_trace(go.Scatter(x=df_cumsum.index, y=df_cumsum['twoii'], mode='lines', name='TWOII'), row=5, col=1)
fig.add_trace(go.Scatter(x=df_cumsum.index, y=df_cumsum['diff'], mode='lines', name='DIFF', yaxis="y2"), row=5, col=1)

# Update layout
fig.update_layout(
    title=symbol,
    template="plotly_white",
    height=1800
)

# Show the plot and save it as an HTML file
fig.write_html("combined_plot.html")
fig.show()
